In [21]:
import svfpy
import rasterio

In [18]:
mds = '../test/data/mds_sao_paulo_city_4000.tiff'

In [19]:
seila = svfpy.SVF(mds_path=mds, kernel_size=11)

In [20]:
seila.view_as_windows.dtype

dtype('float16')

In [34]:
seila.mds_src.transform, rasterio.pad(seila.mds, seila.mds_src.transform, 5, mode='reflect')[1]

(Affine(0.5, 0.0, 331644.76279623254,
        0.0, -0.5, 7391573.901147625),
 Affine(0.5, 0.0, 331642.26279623254,
        0.0, -0.5, 7391576.401147625))

In [33]:
rasterio.pad(seila.mds, seila.mds_src.transform, 5, mode='reflect')[0].shape

(4010, 4010)

In [5]:
import numpy as np

In [9]:
it = np.nditer(seila.mds, flags=['multi_index'])
total = 0
for i in it:
    total += i
    if it.multi_index[0] % 1000 == 0 and it.multi_index[1] % 1000 == 0:
        print(it.multi_index)

(0, 0)
(0, 1000)
(0, 2000)
(0, 3000)
(1000, 0)
(1000, 1000)
(1000, 2000)
(1000, 3000)
(2000, 0)
(2000, 1000)
(2000, 2000)
(2000, 3000)
(3000, 0)
(3000, 1000)
(3000, 2000)
(3000, 3000)


In [24]:
total

12692545727.5

In [13]:
rays = seila.rays()

In [14]:
rays.shape

(3990, 3990, 40, 6)

In [15]:
rays = np.amax(rays, axis=3)

In [16]:
rays.shape

(3990, 3990, 40)

In [102]:
a = np.array([[0,1,0,1],[2,3,2,3],[0,1,0,1],[2,3,2,3]])
a.shape

(4, 4)

In [103]:
np.repeat(a, 4)

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3,
       3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3])

In [89]:
from skimage.transform import resize

In [95]:
np.resize(a, (2,8))

array([[0, 1, 2, 3, 0, 1, 2, 3],
       [0, 1, 2, 3, 0, 1, 2, 3]])

In [91]:
resize(a, (4,4), preserve_range=True)

array([[0.75, 0.75, 1.25, 1.25],
       [0.75, 0.75, 1.25, 1.25],
       [1.75, 1.75, 2.25, 2.25],
       [1.75, 1.75, 2.25, 2.25]])

In [78]:
np.tile(np.repeat(a, 4), (2,2))

array([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1,
        1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
       [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1,
        1, 1, 2, 2, 2, 2, 3, 3, 3, 3]])

In [76]:
np.tile(a, (2,2))

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [0, 1, 0, 1],
       [2, 3, 2, 3]])

In [74]:
np.repeat(a.reshape(4,1), 4).reshape(8,2)

array([[0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [2, 2],
       [2, 2],
       [3, 3],
       [3, 3]])

In [60]:
np.repeat(a, [4,4], axis=1)

array([[0, 0, 0, 0, 1, 1, 1, 1],
       [2, 2, 2, 2, 3, 3, 3, 3]])